# **installation de la bibliotheque NEO4j**

In [1]:
! pip install neo4j
! pip install sqlite3
! pip install os


  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.28.1-py3-none-any.whl (312 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for sqlite3
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for os


# Migration de données du modèle relationnel vers la base postgres

In [ ]:
import os
import sqlite3
from neo4j import GraphDatabase

# Fonction pour se connecter à SQLite avec un chemin absolu
def connect_to_sqlite(db_name="infractions.db"):
    db_path = os.path.abspath(db_name)
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Le fichier {db_name} est introuvable.")
    return sqlite3.connect(db_path)

# Fonction pour se connecter à Neo4j
def connect_to_neo4j():
    uri = input("Entrez l'URI de Neo4j (par défaut: bolt://localhost:7687): ") or "bolt://localhost:7687"
    user = input("Entrez le nom d'utilisateur Neo4j (par défaut: neo4j): ") or "neo4j"
    password = input("Entrez le mot de passe Neo4j: ")
    return GraphDatabase.driver(uri, auth=(user, password))

# Fonction pour insérer plusieurs nœuds en batch
def create_nodes(tx, label, data_list):
    if not data_list:
        return
    query = f"""
    UNWIND $data AS row
    CREATE (n:{label})
    SET n = row
    """
    tx.run(query, data=data_list)

# Fonction pour insérer plusieurs relations en batch
def create_relationships(tx, start_label, start_key, end_label, end_key, rel_type, data_list):
    if not data_list:
        return
    query = f"""
    UNWIND $data AS row
    MATCH (a:{start_label} {{ {start_key}: row.{start_key} }})
    MATCH (b:{end_label} {{ {end_key}: row.{end_key} }})
    CREATE (a)-[:{rel_type}]->(b)
    """
    tx.run(query, data=data_list)

# Fonction pour insérer les faits et créer les relations associées
def create_fait_relationships(tx, data_list):
    if not data_list:
        return
    query = f"""
    UNWIND $data AS row
    MATCH (b:Brigade {{ id_brigade: row.id_brigade }})
    MATCH (i:Infraction {{ code_infr: row.code_infr }})
    MATCH (d:Departement {{ code_dep: row.code_dep }})
    CREATE (f:Fait {{
        annee: row.annee,
        nb_infraction: row.nb_infraction,
        force_odre: row.force_odre
    }})
    CREATE (b)-[:A_ENREGISTRÉ]->(f)
    CREATE (f)-[:CONCERNE]->(i)
    CREATE (f)-[:DANS]->(d)
    """
    tx.run(query, data=data_list)

# Fonction principale de migration
def migrate_data(sqlite_conn, neo4j_driver, batch_size=10000):
    sqlite_cursor = sqlite_conn.cursor()
    with neo4j_driver.session() as session:

        # Migration des infractions
        print("➡ Migration des infractions...")
        sqlite_cursor.execute("SELECT code_infr, lib_infr FROM infractions")
        batch = [{"code_infr": row[0], "lib_infr": row[1]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Infraction", batch)
        print("✅ Infractions migrées.")

        # Migration des départements
        print("➡ Migration des départements...")
        sqlite_cursor.execute("SELECT code_dep, nom_dep FROM departements")
        batch = [{"code_dep": row[0], "nom_dep": row[1]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Departement", batch)
        print("✅ Départements migrés.")

        # Migration des forces de l'ordre
        print("➡ Migration des forces de l'ordre...")
        sqlite_cursor.execute("SELECT force_odre FROM force_odre")
        batch = [{"force_odre": row[0]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "ForceOrdre", batch)
        print("✅ Forces de l'ordre migrées.")

        # Migration des brigades
        print("➡ Migration des brigades...")
        sqlite_cursor.execute("SELECT id_brigade, lib_brigade, lib_perimetre, code_dep, force_odre FROM brigade")
        batch = [{"id_brigade": row[0], "lib_brigade": row[1], "lib_perimetre": row[2], "code_dep": row[3], "force_odre": row[4]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_nodes, "Brigade", batch)
        print("✅ Brigades migrées.")

        # Création des relations Brigade ↔ Département
        print("➡ Création des relations Brigade ↔ Département...")
        session.execute_write(create_relationships, "Brigade", "code_dep", "Departement", "code_dep", "APPARTIENT_A", batch)
        print("✅ Relations créées.")

        # Création des relations Brigade ↔ Force de l'ordre
        print("➡ Création des relations Brigade ↔ Force de l'ordre...")
        session.execute_write(create_relationships, "Brigade", "force_odre", "ForceOrdre", "force_odre", "FAIT_PARTIE_DE", batch)
        print("✅ Relations créées.")

        # Migration des faits et relations Brigade ↔ Fait ↔ Infraction
        print("➡ Migration des faits (infractions enregistrées)...")
        sqlite_cursor.execute("SELECT code_infr, id_brigade, annee, nb_infraction, code_dep, force_odre FROM fait")
        batch = [{"code_infr": row[0], "id_brigade": row[1], "annee": row[2], "nb_infraction": row[3], "code_dep": row[4], "force_odre": row[5]} for row in sqlite_cursor.fetchall()]
        session.execute_write(create_fait_relationships, batch)
        print("✅ Faits migrés avec succès.")

# Point d'entrée du script
if __name__ == "__main__":
    sqlite_conn = None
    neo4j_driver = None

    try:
        sqlite_conn = connect_to_sqlite()
        print("✅ Connexion à SQLite réussie.")

        neo4j_driver = connect_to_neo4j()
        print("✅ Connexion à Neo4j réussie.")

        print("🚀 Début de la migration des données...")
        migrate_data(sqlite_conn, neo4j_driver)
        print("🎉 Migration terminée avec succès.")

    except Exception as e:
        print(f"❌ Erreur : {e}")

    finally:
        if sqlite_conn is not None:
            sqlite_conn.close()
        if neo4j_driver is not None:
            neo4j_driver.close()


✅ Connexion à SQLite réussie.
✅ Connexion à Neo4j réussie.
🚀 Début de la migration des données...
➡ Migration des infractions...
✅ Infractions migrées.
➡ Migration des départements...
✅ Départements migrés.
➡ Migration des forces de l'ordre...
✅ Forces de l'ordre migrées.
➡ Migration des brigades...
✅ Brigades migrées.
➡ Création des relations Brigade ↔ Infraction...
✅ Relations Brigade ↔ Infraction créées.
🎉 Migration terminée avec succès.
